#### Credit Scoring and Segmentation 

In [ ]:
import pandas as pd
    
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

data = pd.read_csv("D://Files//credit_scoring.csv")
print(data.head())

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
#the distribution of the credit utilization ratio in the data:


credit_utilization_fig = px.box(data, y='Credit Utilization Ratio',

                                title='Credit Utilization Ratio Distribution')

credit_utilization_fig.show()

In [ ]:
# the distribution of the loan amount in the data:

loan_amount_fig = px.histogram(data, x='Loan Amount', 
                          nbins=20, 

                               title='Loan Amount Distribution')

loan_amount_fig.show()

In [ ]:
#the correlation in the data:


numeric_df = data[['Credit Utilization Ratio', 

                   'Payment History', 

                   'Number of Credit Accounts', 

                   'Loan Amount', 'Interest Rate', 

                   'Loan Term']]

correlation_fig = px.imshow(numeric_df.corr(), 

                            title='Correlation Heatmap')

correlation_fig.show()

In [ ]:
#Calculating Credit Scores

In [ ]:
# Define the mapping for categorical features
education_level_mapping = {'High School': 1, 'Bachelor': 2, 'Master': 3, 'PhD': 4}
employment_status_mapping = {'Unemployed': 0, 'Employed': 1, 'Self-Employed': 2}

# Apply mapping to categorical features
data['Education Level'] = data['Education Level'].map(education_level_mapping)
data['Employment Status'] = data['Employment Status'].map(employment_status_mapping)

# Calculate credit scores using the complete FICO formula
credit_scores = []

for index, row in data.iterrows():
    payment_history = row['Payment History']
    credit_utilization_ratio = row['Credit Utilization Ratio']
    number_of_credit_accounts = row['Number of Credit Accounts']
    education_level = row['Education Level']
    employment_status = row['Employment Status']

    # Apply the FICO formula to calculate the credit score
    credit_score = (payment_history * 0.35) + (credit_utilization_ratio * 0.30) + (number_of_credit_accounts * 0.15) + (education_level * 0.10) + (employment_status * 0.10)
    credit_scores.append(credit_score)

# Add the credit scores as a new column to the DataFrame
data['Credit Score'] = credit_scores

print(data.head())

Firstly, it defines mappings for two categorical features:

“Education Level” and “Employment Status”. The “Education Level” mapping assigns numerical values to different levels of education, such as “High School” being mapped to 1, “Bachelor” to 2, “Master” to 3, and “PhD” to 4. 

The “Employment Status” mapping assigns numerical values to different employment statuses, such as “Unemployed” being mapped to 0, “Employed” to 1, and “Self-Employed” to 2.

Next, the code applies the defined mappings to the corresponding columns in the DataFrame. It transforms the values of the “Education Level” and “Employment Status” columns from their original categorical form to the mapped numerical representations.

After that, the code initiates an iteration over each row of the DataFrame to calculate the credit scores for each individual. It retrieves the values of relevant features, such as “Payment History”, “Credit Utilization Ratio”, “Number of Credit Accounts”, “Education Level”, and “Employment Status”, from each row.

### Segmentation Based on Credit Scores

In [ ]:
#use the KMeans clustering algorithm to segment customers based on their credit scores:
from sklearn.cluster import KMeans

X = data[['Credit Score']]
kmeans = KMeans(n_clusters=4, n_init=10, random_state=42)
kmeans.fit(X)
data['Segment'] = kmeans.labels_

In [ ]:
#  The segments:
# Convert the 'Segment' column to category data type
data['Segment'] = data['Segment'].astype('category')

# Visualize the segments using Plotly
fig = px.scatter(data, x=data.index, y='Credit Score', color='Segment',
                 color_discrete_sequence=['green', 'blue', 'yellow', 'red'])
fig.update_layout(
    xaxis_title='Customer Index',
    yaxis_title='Credit Score',
    title='Customer Segmentation based on Credit Scores'
)
fig.show()

In [ ]:
# name the segments based on the above clusters 

data['Segment'] = data['Segment'].map({2: 'Very Low', 
                                       0: 'Low',
                                       1: 'Good',
                                       3: "Excellent"})

# Convert the 'Segment' column to category data type
data['Segment'] = data['Segment'].astype('category')

# Visualize the segments using Plotly
fig = px.scatter(data, x=data.index, y='Credit Score', color='Segment',
                 color_discrete_sequence=['green', 'blue', 'yellow', 'red'])
fig.update_layout(
    xaxis_title='Customer Index',
    yaxis_title='Credit Score',
    title='Customer Segmentation based on Credit Scores'
)
fig.show()

In [ ]:
data[['Segment','Age','Gender']]

In [ ]:
def age(x):
    if x<=30:
        return '0-30'
    elif 31<x<=50:
        return '31-50'
    else:
        return 'above 50'
data['age_bins']=data['Age'].map(age)

In [ ]:
data[['Segment','age_bins','Gender']]

In [ ]:
data.query('Segment=="Excellent" & age_bins=="above 50"')[['Segment','age_bins','Gender']].shape